<a href="https://colab.research.google.com/github/WideSu/Python-for-DS/blob/main/Ray_HyperParam_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TO-DO
- [x] Test the average time usage and RMSE for each epoch using scikit-learn random search
- [ ] Test TPE hyper param tuning for HyperOpt, Ray, Optuna
- [ ] Plot the RMSE through timeline
- [ ] Use the different sampler in Optuna: Random,TPE,CMA-ES,NSGA-II

The outcome:
- A chart consisting the average RMSE and excuation time for all hyper parameter tunning methods

|HPO Package                                  |Avg RMSE                        |Avg Time Ellipsed                                            |
|---------------------------------------------|--------------------------------|-------------------------------------------------------------|
|Scikit-learn                                 |                                |                                                             |
|HyperOpt                                     |                                |                                                             |
|Ray                                          |                                |                                                             |
|Optuna                                       |                                |                                                             |


In [1]:
# @title Mont on Google Drive
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/HPO/

Mounted at /content/drive
/content/drive/MyDrive/HPO


In [3]:
# @title Install and import packages
! pip install dateutil
! pip install lightgbm
! pip install ray
import pandas as pd
import dateutil
import datetime
import ray
from tqdm import tqdm, trange
from lightgbm import LGBMRegressor
import sklearn
import math
import time

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement dateutil (from versions: none)
ERROR: No matching distribution found for dateutil
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# @title Read-in data and check data type and volume
df = pd.read_csv('./exp_data.csv')
df.info()

In [5]:
# @title Change into datatime type
df[["date"]] = df[["date"]].apply(pd.to_datetime)

In [ ]:
from ray.tune.suggest.basic_variant import BasicVariantGenerator
# @title Configuration for experiements
library_evaluation_df = {
    'Library' : [],
    'Train Start Date': [],
    'Train End Date': [],
    'Test Start Date': [],
    'Test End Date': [],
    'Smallest RMSE': [],
    'Time Ellipsed': []
}

n_trials = 15 # @param run 15 times of hyper-param tuning

train_timespan_months = 180 # 5 years
whole_period_months = 60
test_timespan_months = 1

first_end_time = datetime.datetime(2015, 12, 1)
feat_cols = ['absacc', 'acc', 'age', 'agr', 'baspread','bm', 'bm_ia',
             'cash', 'cashdebt', 'cashpr', 'cfp', 'cfp_ia', 'chatoia', 'chcsho', 'chempia', 'chinv', 'chmom',
             'chpmia', 'chtx', 'cinvest', 'convind', 'currat', 'depr', 'divi', 'divo', 'dolvol', 'dy', 
             'egr', 'ep', 'gma', 'grcapx', 'grltnoa', 'herf', 'hire', 'ill', 'indmom', 'invest', 'lev', 'lgr',
             'maxret', 'mom12m', 'mom1m', 'mom36m', 'mom6m', 'ms', 'mve_ia', 'mvel1', 'nincr', 'operprof',
             'orgcap', 'pchcapx_ia', 'pchcurrat', 'pchdepr', 'pchgm_pchsale', 'pchquick', 'pchsale_pchinvt',
             'pchsale_pchrect', 'pchsale_pchxsga', 'pchsaleinv', 'pctacc', 'ps', 'quick', 'rd', 'rd_mve',
             'rd_sale', 'realestate', 'retvol', 'roaq', 'roavol', 'roeq', 'roic', 'rsup', 'salecash', 'pricedelay',
             'saleinv', 'salerec', 'secured', 'securedind', 'sgr', 'sin', 'sp', 'std_dolvol', 'std_turn',
             'stdacc', 'stdcf', 'tang', 'tb', 'turn', 'zerotrade','aeavol','ear','beta','betasq','idiovol']
y_col = 'ret'
train_end_date = first_end_time
time_usage = []
score_list = []
timeline = []

# Evaluation details for each train and test timespan
evaluate_detail_df = {
    'Train Start Date': [],
    'Train End Date': [],
    'Test Start Date': [],
    'Test End Date': [],
    'Smallest RMSE': [],
    'Time Ellipsed': []
}

search_space = {
    'n_estimators': ray.tune.randint(50,501),
    "num_leaves":  ray.tune.randint(10,513),
    'min_data_in_leaf': ray.tune.randint(10, 81),
    'bagging_fraction': ray.tune.uniform( 0.0, 1.0),
    'learning_rate': ray.tune.uniform( 0.01, 0.1),
    'lambda_l1': ray.tune.uniform( 0.01, 1),
    'lambda_l2': ray.tune.uniform( 0.01, 1),
}

# Uncomment this to enable distributed execution
# `ray.init(address="auto")`

def objective(config):    
    param_dict = dict(
            n_estimators = int(config['n_estimators']),
            num_leaves = int(config['num_leaves']),
            min_data_in_leaf = int(config['min_data_in_leaf']),
            bagging_fraction = config['bagging_fraction'],
            learning_rate = config['learning_rate'],
            lambda_l1 = config['lambda_l1'],
            lambda_l2 = config['lambda_l2']
        )
    model = LGBMRegressor(seed=42, **param_dict)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = sklearn.metrics.mean_squared_error(y_test, y_pred)
    rmse = math.sqrt(mse)
    return rmse  # An objective value linked with the Trial object.
# @title Back-test for 5 years
predict_times = 60
for period_time in trange(predict_times):
    train_start_date = train_end_date - dateutil.relativedelta.relativedelta(months=train_timespan_months)
    test_end_date = train_end_date + dateutil.relativedelta.relativedelta(months=test_timespan_months)
    print(train_start_date, train_end_date, test_end_date)
    train_data = df.query(f'"{train_start_date}" < date <= "{train_end_date}"')
    test_data = df.query(f'"{train_end_date}" < date <= "{test_end_date}"')
    X_train = train_data[feat_cols].values
    y_train = train_data[y_col].values
    X_test = test_data[feat_cols].values
    y_test = test_data[y_col].values.ravel()
    
    # run a trial
    ts = time.time()
    analysis = ray.tune.run(
        objective, 
        mode = 'min',
        search_alg = BasicVariantGenerator(
            constant_grid_search = False,
            random_state = 0
        ),
        num_samples = n_trials,
        config=search_space)
    te = time.time()

    exc_time = te-ts
    best_score = analysis.best_result['_metric']
    evaluate_detail_df['Smallest RMSE'].append(best_score)
    evaluate_detail_df['Time Ellipsed'].append(exc_time)
    evaluate_detail_df['Train Start Date'].append(train_start_date)
    evaluate_detail_df['Train End Date'].append(train_end_date)
    evaluate_detail_df['Test Start Date'].append(train_end_date+dateutil.relativedelta.relativedelta(months=1))
    evaluate_detail_df['Test End Date'].append(test_end_date)
    train_end_date += dateutil.relativedelta.relativedelta(months=1)
evaluate_detail_df = pd.DataFrame(evaluate_detail_df)


  0%|          | 0/60 [00:00<?, ?it/s]

2000-12-01 00:00:00 2015-12-01 00:00:00 2016-01-01 00:00:00


2022-06-05 03:50:20,058	WARNING callback.py:126 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`
2022-06-05 03:50:28,581	INFO trial_runner.py:803 -- starting objective_9e50d_00000
2022-06-05 03:50:29,238	WARNING worker.py:1382 -- Warning: The actor ImplicitFunc is very large (38 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.
2022-06-05 03:50:30,070	WARNING util.py:171 -- The `start_trial` operation took 1.481 s, which may be a performance bottleneck.


Trial name,status,loc,bagging_fraction,lambda_l1,lambda_l2,learning_rate,min_data_in_leaf,n_estimators,num_leaves
objective_9e50d_00000,RUNNING,172.28.0.2:1964,0.0409735,0.815138,0.913628,0.0114875,46,433,330
objective_9e50d_00001,PENDING,,0.729497,0.935722,0.817695,0.0589262,53,171,263
objective_9e50d_00002,PENDING,,0.0335856,0.183899,0.864547,0.075669,37,352,11
objective_9e50d_00003,PENDING,,0.299712,0.0380365,0.13304,0.0480418,48,436,21
objective_9e50d_00004,PENDING,,0.615385,0.997238,0.981027,0.044531,47,53,347
objective_9e50d_00005,PENDING,,0.650459,0.395032,0.143746,0.0719602,58,341,200
objective_9e50d_00006,PENDING,,0.310242,0.890593,0.934703,0.0537252,70,311,372
objective_9e50d_00007,PENDING,,0.57153,0.598357,0.344532,0.0389682,35,286,277
objective_9e50d_00008,PENDING,,0.227158,0.0931752,0.834318,0.0660868,33,393,206
objective_9e50d_00009,PENDING,,0.239369,0.0679824,0.342756,0.0888836,65,451,211


2022-06-05 03:50:36,364	INFO trial_runner.py:803 -- starting objective_9e50d_00001


...

Trial name,status,loc,bagging_fraction,lambda_l1,lambda_l2,learning_rate,min_data_in_leaf,n_estimators,num_leaves,iter,total time (s),_metric
objective_9e50d_00000,RUNNING,172.28.0.2:1964,0.0409735,0.815138,0.913628,0.0114875,46,433,330,,,
objective_9e50d_00002,RUNNING,172.28.0.2:2094,0.0335856,0.183899,0.864547,0.075669,37,352,11,,,
objective_9e50d_00003,PENDING,,0.299712,0.0380365,0.13304,0.0480418,48,436,21,,,
objective_9e50d_00004,PENDING,,0.615385,0.997238,0.981027,0.044531,47,53,347,,,
objective_9e50d_00005,PENDING,,0.650459,0.395032,0.143746,0.0719602,58,341,200,,,
objective_9e50d_00006,PENDING,,0.310242,0.890593,0.934703,0.0537252,70,311,372,,,
objective_9e50d_00007,PENDING,,0.57153,0.598357,0.344532,0.0389682,35,286,277,,,
objective_9e50d_00008,PENDING,,0.227158,0.0931752,0.834318,0.0660868,33,393,206,,,
objective_9e50d_00009,PENDING,,0.239369,0.0679824,0.342756,0.0888836,65,451,211,,,
objective_9e50d_00010,PENDING,,0.796324,0.0615011,0.410506,0.0307578,71,308,85,,,


Trial name,status,loc,bagging_fraction,lambda_l1,lambda_l2,learning_rate,min_data_in_leaf,n_estimators,num_leaves,iter,total time (s),_metric
objective_9e50d_00000,RUNNING,172.28.0.2:1964,0.0409735,0.815138,0.913628,0.0114875,46,433,330,,,
objective_9e50d_00002,RUNNING,172.28.0.2:2094,0.0335856,0.183899,0.864547,0.075669,37,352,11,,,
objective_9e50d_00003,PENDING,,0.299712,0.0380365,0.13304,0.0480418,48,436,21,,,
objective_9e50d_00004,PENDING,,0.615385,0.997238,0.981027,0.044531,47,53,347,,,
objective_9e50d_00005,PENDING,,0.650459,0.395032,0.143746,0.0719602,58,341,200,,,
objective_9e50d_00006,PENDING,,0.310242,0.890593,0.934703,0.0537252,70,311,372,,,
objective_9e50d_00007,PENDING,,0.57153,0.598357,0.344532,0.0389682,35,286,277,,,
objective_9e50d_00008,PENDING,,0.227158,0.0931752,0.834318,0.0660868,33,393,206,,,
objective_9e50d_00009,PENDING,,0.239369,0.0679824,0.342756,0.0888836,65,451,211,,,
objective_9e50d_00010,PENDING,,0.796324,0.0615011,0.410506,0.0307578,71,308,85,,,


Trial name,status,loc,bagging_fraction,lambda_l1,lambda_l2,learning_rate,min_data_in_leaf,n_estimators,num_leaves,iter,total time (s),_metric
objective_9e50d_00000,RUNNING,172.28.0.2:1964,0.0409735,0.815138,0.913628,0.0114875,46,433,330,,,
objective_9e50d_00002,RUNNING,172.28.0.2:2094,0.0335856,0.183899,0.864547,0.075669,37,352,11,,,
objective_9e50d_00003,PENDING,,0.299712,0.0380365,0.13304,0.0480418,48,436,21,,,
objective_9e50d_00004,PENDING,,0.615385,0.997238,0.981027,0.044531,47,53,347,,,
objective_9e50d_00005,PENDING,,0.650459,0.395032,0.143746,0.0719602,58,341,200,,,
objective_9e50d_00006,PENDING,,0.310242,0.890593,0.934703,0.0537252,70,311,372,,,
objective_9e50d_00007,PENDING,,0.57153,0.598357,0.344532,0.0389682,35,286,277,,,
objective_9e50d_00008,PENDING,,0.227158,0.0931752,0.834318,0.0660868,33,393,206,,,
objective_9e50d_00009,PENDING,,0.239369,0.0679824,0.342756,0.0888836,65,451,211,,,
objective_9e50d_00010,PENDING,,0.796324,0.0615011,0.410506,0.0307578,71,308,85,,,


Result for objective_9e50d_00002:
  _metric: 0.12249702683110897
  date: 2022-06-05_03-51-59
  done: false
  experiment_id: 6ea689f2df2b4d6fb6114efca8ee67e4
  hostname: 220efc72785a
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  pid: 2094
  time_since_restore: 13.945075035095215
  time_this_iter_s: 13.945075035095215
  time_total_s: 13.945075035095215
  timestamp: 1654401119
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 9e50d_00002
  warmup_time: 0.007077693939208984
  
Result for objective_9e50d_00002:
  _metric: 0.12249702683110897
  date: 2022-06-05_03-51-59
  done: true
  experiment_id: 6ea689f2df2b4d6fb6114efca8ee67e4
  experiment_tag: 2_bagging_fraction=0.033586,lambda_l1=0.1839,lambda_l2=0.86455,learning_rate=0.075669,min_data_in_leaf=37,n_estimators=352,num_leaves=11
  hostname: 220efc72785a
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  pid: 2094
  time_since_restore: 13.945075035095215
  time_this_iter_s: 13.945075035095215
  time_total_s: 13.

2022-06-05 03:52:00,994	INFO trial_runner.py:803 -- starting objective_9e50d_00003


In [ ]:
library_evaluation_df['Library'].extend(['Ray' for _ in range(len(evaluate_detail_df))])
library_evaluation_df['Train Start Date'].extend(evaluate_detail_df['Train Start Date'])
library_evaluation_df['Train End Date'].extend(evaluate_detail_df['Train End Date'])
library_evaluation_df['Test Start Date'].extend(evaluate_detail_df['Test Start Date'])
library_evaluation_df['Test End Date'].extend(evaluate_detail_df['Test End Date'])
library_evaluation_df['Smallest RMSE'].extend(evaluate_detail_df['Smallest RMSE'])
library_evaluation_df['Time Ellipsed'].extend(evaluate_detail_df['Time Ellipsed'])

In [ ]:
ray_tune_res = pd.DataFrame(library_evaluation_df)
ray_tune_res.to_pickle('ray_tune_res.pkl')

,Library,Train Start Date,Train End Date,Test Start Date,Test End Date,Smallest RMSE,Time Ellipsed
0,Optuna,2000-12-01,2015-12-01,2016-01-01,2016-01-01,0.118880,800.256522
1,Optuna,2001-01-01,2016-01-01,2016-02-01,2016-02-01,0.067029,499.085326
2,Optuna,2001-02-01,2016-02-01,2016-03-01,2016-03-01,0.091569,536.559700
3,Optuna,2001-03-01,2016-03-01,2016-04-01,2016-04-01,0.072515,502.082866
4,Optuna,2001-04-01,2016-04-01,2016-05-01,2016-05-01,0.055176,806.044957
5,Optuna,2001-05-01,2016-05-01,2016-06-01,2016-06-01,0.066578,724.420032
6,Optuna,2001-06-01,2016-06-01,2016-07-01,2016-07-01,0.065813,418.449212
7,Optuna,2001-07-01,2016-07-01,2016-08-01,2016-08-01,0.063775,621.808052
8,Optuna,2001-08-01,2016-08-01,2016-09-01,2016-09-01,0.051464,479.586529
9,Optuna,2001-09-01,2016-09-01,2016-10-01,2016-10-01,0.076580,563.586720


In [ ]:
ray_tune_res